In [39]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.python.keras.layers import Dense, LSTM, RepeatVector, Flatten
from tensorflow.keras.layers import TimeDistributed
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
week = "/home/lucien/Bureau/projet_ml_finance/datahub/wti-week_csv.csv"
daily = "/home/lucien/Bureau/projet_ml_finance/datahub/wti-daily_csv.csv"

data_week = pd.read_csv(week, delimiter = ',')
data_daily = pd.read_csv(daily, delimiter = ',')

#format datime
data_week['Date'] = pd.to_datetime(data_week['Date'])

#on complète les valeurs de Price null avec la méthode forward fill

data_week['Price'].fillna(method='ffill', inplace=True)



/tmp/ipykernel_16498/545267995.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_week['Price'].fillna(method='ffill', inplace=True)


In [41]:
#let's split data
data = data_daily['Price']
split_index = int(len(data) * 0.8)

# Diviser les données en train et test
X_train = data[:split_index]
X_test = data[split_index:]

split_index = int(len(X_test) * 0.8)
X_val= X_test[:split_index]
X_test = X_test[split_index:]

In [42]:
# Supposons que X_train et X_test sont vos séries temporelles
# Création des échantillons et des étiquettes pour l'entraînement
def create_samples(series, lookback=30, horizon=5):
    samples, labels = [], []
    for i in range(len(series) - lookback - horizon + 1):
        samples.append(series[i : i + lookback])  # Séquences temporelles précédentes (caractéristiques)
        labels.append(series[i + lookback : i + lookback + horizon])  # Valeurs futures à prédire (étiquettes)
    return np.array(samples), np.array(labels)

lookback = 200  # Nombre de pas de temps précédents à utiliser comme caractéristiques
horizon = 5   # Nombre de pas de temps à prédire dans le futur

X_train_samples, y_train_labels = create_samples(X_train, lookback, horizon)
X_val_samples, y_val_labels = create_samples(X_val, lookback, horizon)
X_test_samples, y_test_labels = create_samples(X_test, lookback, horizon)

print("train shape", X_train_samples.shape)
print("val shape", X_val_samples.shape)
print("test shape", X_test_samples.shape)

train shape (6596, 200)
val shape (1156, 200)
test shape (137, 200)


In [43]:
# #on reshape en ajoutant le nombre de feautures: Reshape from [samples, timesteps] into [samples, timesteps, features].

X_train_series = X_train_samples.reshape((X_train_samples.shape[0], X_train_samples.shape[1], 1))
X_val_series = X_val_samples.reshape((X_val_samples.shape[0], X_val_samples.shape[1], 1))
X_test_series = X_test_samples.reshape((X_test_samples.shape[0], X_test_samples.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_val_series.shape)
print('test set shape', X_test_series.shape)

Train set shape (6596, 200, 1)
Validation set shape (1156, 200, 1)
test set shape (137, 200, 1)


In [44]:
subsequences = 5 #chaque séquence est divisée en plusieurs sous séquences de 2 étapes temporelles chacune, le CNN va d'abord extraire les features de chacune des sous séries

timesteps = X_train_series.shape[1]//subsequences
X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, timesteps, 1))
X_val_series_sub = X_val_series.reshape((X_val_series.shape[0], subsequences, timesteps, 1))
X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, timesteps, 1))
print('Train set shape', X_train_series_sub.shape)
print('Validation set shape', X_val_series_sub.shape)
print('test set shape', X_test_series_sub.shape)

Train set shape (6596, 5, 40, 1)
Validation set shape (1156, 5, 40, 1)
test set shape (137, 5, 40, 1)


In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam

epochs = 40
batch = 25
lr = 0.003
adam = optimizers.Adam(lr)

model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer=adam)


In [46]:
cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, y_train_labels, validation_data=(X_val_series_sub, y_val_labels), epochs=40, verbose=2)

Epoch 1/40
207/207 - 3s - loss: 133.6597 - val_loss: 74.2777 - 3s/epoch - 14ms/step
Epoch 2/40
207/207 - 1s - loss: 40.3066 - val_loss: 40.7112 - 1s/epoch - 5ms/step
Epoch 3/40
207/207 - 1s - loss: 17.5141 - val_loss: 18.4274 - 1s/epoch - 5ms/step
Epoch 4/40
207/207 - 2s - loss: 12.0816 - val_loss: 49.6258 - 2s/epoch - 7ms/step
Epoch 5/40
207/207 - 1s - loss: 9.5100 - val_loss: 7.7236 - 1s/epoch - 7ms/step
Epoch 6/40
207/207 - 1s - loss: 8.0492 - val_loss: 30.8115 - 1s/epoch - 5ms/step
Epoch 7/40
207/207 - 1s - loss: 7.8938 - val_loss: 7.4433 - 1s/epoch - 5ms/step
Epoch 8/40
207/207 - 1s - loss: 8.7567 - val_loss: 7.9872 - 1s/epoch - 5ms/step
Epoch 9/40
207/207 - 1s - loss: 6.8397 - val_loss: 8.8842 - 1s/epoch - 5ms/step
Epoch 10/40
207/207 - 1s - loss: 6.2868 - val_loss: 18.2019 - 1s/epoch - 5ms/step
Epoch 11/40
207/207 - 1s - loss: 7.2155 - val_loss: 16.8537 - 1s/epoch - 5ms/step
Epoch 12/40
207/207 - 1s - loss: 6.4716 - val_loss: 9.5169 - 1s/epoch - 5ms/step
Epoch 13/40
207/207 - 1s

In [47]:
# Faire des prédictions sur l'ensemble de test
y_pred = model_cnn_lstm.predict(X_test_series_sub)

# Fonction pour calculer la Mean Percentage Error (MPE)
def mean_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calcul de la MPE sur l'ensemble de test
mpe = mean_percentage_error(y_test_labels, y_pred)
print(f"Mean Percentage Error on Test Set: {mpe:.2f}%")


5/5 [==============================] - 0s 2ms/step
Mean Percentage Error on Test Set: 3.42%


Résultats:

-3,55% pour un CNN-LSTM avec 2 pour hyperparamètres des subsequence
-3% pour un CNN-LSTM avec 5 pour hyperparamètres des subsequence (mais temps d'entrainement plus long)
-3,46% pour 10

Remarque: meilleur performance car en réduisant la taille de la sous séquence on capture des motifs temporels plus fins et plus complexes. Cependant si on augmente trop on risque de louper des motifs important pour la prédiciton donc attention à ne pas trop segmenter en petites sous séquences. Par exemple quand on choisit 10 cela signifie qu'on étudie que les motifs de taille 3 ce qui peut être trop limité dans le cadre de la prédiction de l'évolution du prix de crude oil.